In [1]:
import numpy as np
import pandas as pd
import h5py
import json
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import GradScaler, autocast
from sklearn.metrics import confusion_matrix, classification_report
from tqdm import tqdm
import gc
import time
import warnings
warnings.filterwarnings('ignore')

# Check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🚀 Using device: {device}")

if device.type == 'cuda':
    print(f"GPU: {torch.cuda.get_device_name()}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f}GB")
    torch.cuda.empty_cache()

print("✅ Imports completed successfully!")

🚀 Using device: cuda
GPU: NVIDIA GeForce RTX 5070 Ti
GPU Memory: 15.9GB
✅ Imports completed successfully!


In [2]:
# Method 1: Import your existing model file
try:
    # Option A: Run your model script directly
    %run ./CNN_LSTM_parallel.py
    print("✅ Successfully imported your CNN_LSTM_parallel.py")
    
except Exception as e:
    print(f"⚠️ Could not run CNN_LSTM_parallel.py: {e}")
    print("💡 Using backup model definition...")
    
    # Option B: Simple backup model definition (compatible with your original)
    class CNN_LSTM_Parallel(nn.Module):
        def __init__(self, 
                     input_channels=2,
                     sequence_length=1024,
                     num_classes=8,
                     cnn_filters=[32, 64, 128],  # Smaller for 4GB VRAM
                     lstm_hidden_dim=64,
                     lstm_num_layers=1,
                     dropout=0.2):
            
            super(CNN_LSTM_Parallel, self).__init__()
            
            self.sequence_length = sequence_length
            self.lstm_hidden_dim = lstm_hidden_dim
            self.num_classes = num_classes
            
            # Simple CNN branch
            self.cnn_kernels = [3, 5]  # Reduced for memory
            self.convs = nn.ModuleList([
                nn.Sequential(
                    nn.Conv1d(input_channels, cnn_filters[0], kernel_size=k, padding=k//2),
                    nn.BatchNorm1d(cnn_filters[0]),
                    nn.ReLU(),
                    nn.Conv1d(cnn_filters[0], cnn_filters[1], kernel_size=k, padding=k//2),
                    nn.BatchNorm1d(cnn_filters[1]),
                    nn.ReLU(),
                    nn.Conv1d(cnn_filters[1], cnn_filters[2], kernel_size=k, padding=k//2),
                    nn.BatchNorm1d(cnn_filters[2]),
                    nn.ReLU(),
                    nn.AdaptiveMaxPool1d(1)
                ) for k in self.cnn_kernels
            ])
            
            # Simple LSTM branch
            self.lstm = nn.LSTM(
                input_size=input_channels,
                hidden_size=lstm_hidden_dim,
                num_layers=lstm_num_layers,
                dropout=dropout if lstm_num_layers > 1 else 0,
                batch_first=True,
                bidirectional=False  # Simplified for memory
            )
            
            self.dropout = nn.Dropout(dropout)
            
            # Calculate feature sizes
            cnn_feature_size = len(self.cnn_kernels) * cnn_filters[2]  # 2 * 64 = 128
            lstm_feature_size = lstm_hidden_dim  # 32
            total_features = cnn_feature_size + lstm_feature_size  # 160
            
            # Simple classifier
            self.classifier = nn.Sequential(
                nn.Linear(total_features, total_features // 2),
                nn.ReLU(),
                nn.Dropout(dropout),
                nn.Linear(total_features // 2, num_classes)
            )
        
        def forward(self, x):
            # CNN branch
            cnn_outputs = []
            for conv in self.convs:
                cnn_out = conv(x).squeeze(-1)
                cnn_outputs.append(cnn_out)
            
            cnn_features = torch.cat(cnn_outputs, dim=1)
            cnn_features = self.dropout(cnn_features)
            
            # LSTM branch
            lstm_input = x.transpose(1, 2)
            lstm_out, (h_n, c_n) = self.lstm(lstm_input)
            lstm_features = h_n[-1]
            lstm_features = self.dropout(lstm_features)
            
            # Combine and classify
            combined_features = torch.cat([cnn_features, lstm_features], dim=1)
            output = self.classifier(combined_features)
            
            return output
    
    print("✅ Backup model definition loaded")

print("🎯 Model class ready for use!")

✅ Successfully imported your CNN_LSTM_parallel.py
🎯 Model class ready for use!


In [3]:
FILE_PATH = "C:\\workarea\\CNN model\\dataset\\radioml2018\\versions\\2\\GOLD_XYZ_OSC.0001_1024.hdf5"
JSON_PATH = 'C:\\workarea\\CNN model\\dataset\\radioml2018\\versions\\2\\classes-fixed.json' 

TARGET_MODULATIONS = [
    'OOK', '4ASK', '8ASK', 'BPSK', 
    'QPSK', '8PSK', '16PSK', '32PSK'
]

BATCH_SIZE = 256 
LEARNING_RATE = 0.003 
NUM_EPOCHS = 100 
NUM_WORKERS = 0 #Temporary check it  

INPUT_CHANNELS = 2 
SEQUENCE_LENGTH = 1024 
NUM_CLASSES = 8  

TRAIN_RATIO = 0.7 
VALID_RATIO = 0.2 
TEST_RATIO = 0.1 

print("📋 Training Parameters:")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Learning rate: {LEARNING_RATE}")
print(f"  Epochs: {NUM_EPOCHS}")
print(f"  Classes: {NUM_CLASSES}")
print(f"  Target modulations: {TARGET_MODULATIONS}")
print(f"  Device: {device}")
print("✅ Parameters set successfully!")

📋 Training Parameters:
  Batch size: 256
  Learning rate: 0.003
  Epochs: 100
  Classes: 8
  Target modulations: ['OOK', '4ASK', '8ASK', 'BPSK', 'QPSK', '8PSK', '16PSK', '32PSK']
  Device: cuda
✅ Parameters set successfully!


In [4]:
def split_dataset(data,modulation_classes,modulations, snrs,
                 target_modulations,mode,target_snrs,train_ratio = 0.8,valid_ratio = 0.2,test_ratio = 0.0, seed = 48):

    X_output = []
    Y_output = []
    Z_output = []

    target_indices = [] 
    for mod in target_modulations: 
        if mod in modulation_classes: 
            idx = modulation_classes.index(mod)
            if np.any(modulations == idx): 
                target_indices.append(idx) 
    print(f"found len {len(target_indices)} valid modulations for {mode} split")

    for mod_idx in target_indices:
        for snr in target_snrs:
            # find samples for this modulation and SNR 
            indices = np.where((modulations == mod_idx) & (snrs == snr)) [0] 
            if len(indices) == 0: 
                continue 
            #shuffle and split
                       
            np.random.shuffle(indices)
            n_samples = len(indices)
            train_end = int(train_ratio * n_samples)
            valid_end = int((train_ratio + valid_ratio) * n_samples)
            
            if mode == 'train': 
                selected_indices = indices[:train_end] 
            elif mode == 'valid': 
                selected_indices = indices[train_end:valid_end]
            elif mode == 'test':
                selected_indices = indices[valid_end:]
            else:
                raise ValueError(f"Unknown mode: {mode}")

            if len(selected_indices) > 0: 
                X_output.append(data[np.sort(selected_indices)])
                Y_output.append(modulations[np.sort(selected_indices)]) 
                Z_output.append(snrs[np.sort(selected_indices)]) 
    if len(X_output) == 0: 
        print(f"No data found for {mode} split !") 
        return np.array([]), np.array([]), np.array([]) 

    #COmbine all the data 
    X_array = np.vstack(X_output)
    Y_array = np.concatenate(Y_output)
    Z_array = np.concatenate(Z_output)

    unique_labels = np.unique(Y_array) 
    for i, label in enumerate(unique_labels): 
        Y_array[Y_array == label] = i 
    print(f"{mode.capitalize()} split: {len(X_array):,} samples") 
    return X_array,Y_array, Z_array
print("Dataset function defined") 

Dataset function defined


In [5]:
class RadioML18Dataset(Dataset): 
    def __init__(self, mode = 'train', target_modulations = None, file_path = None, json_path = None): 
        super (RadioML18Dataset, self).__init__()

        #use global parameter if not provided 
        self.file_path = file_path or FILE_PATH 
        self.json_path = json_path or JSON_PATH 
        self.target_modulations = target_modulations or TARGET_MODULATIONS 

        print(f"loading {mode} dataset .. ") 
        print(f"Target modulation : {self.target_modulations}") 

        #load data files 
        try: 
            self.hdf5_file = h5py.File(self.file_path,'r') 
            self.modulation_classes = json.load(open(self.json_path,'r')) 
            print("File loaded Succesfully")
        except Exception as e: 
            print(f"Error loading files : {e}") 
            raise e 

        #load array 
        self.X = self.hdf5_file['X'] 
        self.Y = np.argmax(self.hdf5_file['Y'], axis = 1) 
        self.Z = self.hdf5_file['Z'][:,0] 
        self.target_snrs = np.unique(self.Z) 

        print(f"Total dataset : {self.X.shape[0]:,} samples and for SNR Range {self.target_snrs.min()} - {self.target_snrs.max()} dB ")

        #split dataset 
        self.X_data,self.Y_data,self.Z_data = split_dataset(
            data = self.X, 
            modulation_classes= self.modulation_classes, 
            modulations=self.Y, 
            snrs = self.Z, 
            target_modulations = self.target_modulations, 
            mode = mode, 
            target_snrs=self.target_snrs, 
            train_ratio = TRAIN_RATIO, 
            valid_ratio = VALID_RATIO, 
            test_ratio =  TEST_RATIO
        )

        if len(self.X_data) == 0: 
            print(f"No data found for {mode} split! ")
            self.hdf5_file.close()
            return 
        
        ### Do IQ correction 
        X_corrected = np.zeros_like(self.X_data)
        X_corrected[:, :, 0] = self.X_data[:, :, 1]
        X_corrected[:, :, 1] = self.X_data[:, :, 0]
        self.X_data = X_corrected 

        self.num_samples = len(self.X_data)
        self.num_classes = len(self.target_modulations)
        self.num_snrs = len(self.target_snrs)

        print(f"Ready!{mode.capitalize()}, Sampels: {self.num_samples}, classes: {self.num_classes}, SNRS: {self.num_snrs}")
        
        #close file to save memory usage 
        self.hdf5_file.close()

    def __len__(self): 
        return len(self.X_data) if hasattr(self,'X_data') else 0 
    
    def __getitem__(self,idx): 
        if len(self.X_data) == 0: 
            raise IndexError("Dataset is empty")
        x = torch.FloatTensor(self.X_data[idx]).transpose(0,1)
        y = torch.LongTensor([self.Y_data[idx]]).squeeze()
        z = torch.LongTensor([self.Z_data[idx]]).squeeze()

        return x,y,z 
print("dataset defined")

dataset defined


In [6]:
def simpel_model():
    print("Creating model ...")

    model = CNN_LSTM_Parallel(
        input_channels= INPUT_CHANNELS,
        sequence_length= SEQUENCE_LENGTH,
        num_classes=NUM_CLASSES,
        cnn_filters=[64,128 , 256] , # smaller filters for 4 gb of vram
        lstm_hidden_dim=128, # SMALLER LSTM
        lstm_num_layers=3, # single layer
        dropout=0.2
    )

    model = model.to(device)

    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

    print(f"Total parameter is {total_params:,} \n Trainable parameter: {trainable_params:,} \n model size: ~{total_params * 4/1024**2:.1f} MB")

    print("testing model ")
    sample_input = torch.randn(1,2,1024).to(device)

    # Penting: Set model ke mode evaluasi untuk pengujian satu sampel
    model.eval() # <--- Tambahkan baris ini

    with torch.no_grad():
        output = model(sample_input)
        print(f" Input shape : {sample_input.shape} \n output shape : {output.shape}, \n output classes : {output.shape[1]}")

    print("model created successfully ")
    return model

In [9]:
class EarlyStopping: 
    def __init__(self,patience = 15, min_delta = 0.001, restore_best_weights = True): 
        self.patience = patience 
        self.min_delta = min_delta 
        self.restore_best_weights = restore_best_weights
        self.best_loss = float('inf') 
        self.counter = 0 
        self.best_weights = None
        
    def __call__(self, val_loss,model):
        if val_loss < self.best_loss - self.min_delta: 
            self.best_loss = val_loss 
            self.counter = 0 
            if self.restore_best_weights:
                self.best_weights = model.state_dict().copy
            else: 
                self.counter += 1 
            if self.counter >= self.patience: 
                if self.restore_best_weights and self.best_weigths is not None: 
                    model.load_state_dict(self.best_weights) 
                return True 
            return False
        

In [10]:
# Test model creation
try:
    model = simpel_model().to('cuda')
    model.eval()
    # Clean up test tensors
    torch.cuda.empty_cache()
    gc.collect()
    
except Exception as e:
    print(f"❌ Error creating model: {e}")
    print("💡 Make sure your CNN_LSTM_parallel.py is compatible")
    raise e

Creating model ...
Total parameter is 871,496 
 Trainable parameter: 871,496 
 model size: ~3.3 MB
testing model 
 Input shape : torch.Size([1, 2, 1024]) 
 output shape : torch.Size([1, 8]), 
 output classes : 8
model created successfully 


In [8]:
def train_one_epoch(model, dataloader, criterion, optimizer, scaler, device): 
    model.train()
    total_loss = 0.0 
    correct = 0
    total = 0 

    progress_bar = tqdm(dataloader, desc="Training", leave=False)

    for batch_idx, (data, target, snr) in enumerate(progress_bar): 
        data = data.to(device, non_blocking=True)  # Fixed typo: non_blockig -> non_blocking
        target = target.to(device, non_blocking=True)

        optimizer.zero_grad()  # Fixed: zero.grad() -> zero_grad()

        with autocast(): 
            output = model(data)
            loss = criterion(output, target)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()
        _, predicted = torch.max(output, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

        accuracy = 100.0 * correct / total 
        # Fixed: Use keyword arguments instead of colon syntax
        progress_bar.set_postfix(
            Loss=f'{loss.item():.4f}',
            Acc=f'{accuracy:.2f}%'
        )

        if batch_idx % 10 == 0: 
            torch.cuda.empty_cache()

    avg_loss = total_loss / len(dataloader)
    accuracy = 100.0 * correct / total 

    return avg_loss, accuracy 

def validate_one_epoch(model, dataloader, criterion, device): 
    model.eval()
    total_loss = 0.0 
    correct = 0 
    total = 0 
    
    with torch.no_grad(): 
        progress_bar = tqdm(dataloader, desc="Validating", leave=False)

        for data, target, snr in progress_bar: 
            data = data.to(device, non_blocking=True)
            target = target.to(device, non_blocking=True)

            with autocast(): 
                output = model(data)
                loss = criterion(output, target)

            total_loss += loss.item()
            _, predicted = torch.max(output, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

            accuracy = 100.0 * correct / total 
            # Fixed: Use keyword arguments
            progress_bar.set_postfix(
                Loss=f'{loss.item():.4f}',
                Acc=f'{accuracy:.2f}%'
            )
    
    avg_loss = total_loss / len(dataloader)
    accuracy = 100.0 * correct / total 

    return avg_loss, accuracy

In [9]:
def plot_training_results(train_losses,val_losses, train_accs, val_accs): 

    fig, (ax1,ax2,ax3) = plt.subplots(1,2,3, fig_size = (12,4))

    #plot losses 

    ax1.plot(train_losses, label = 'Training Loss', color = 'blue')
    ax1.plot(val_losses, label = 'Validating loss', color = 'red')
    ax1.set_title('Training and validaing loss')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.legend()
    ax1.grid(True)

    #Plot Accuracies 
    ax2.plot(train_accs, label='Training Accuracy', color='blue')
    ax2.plot(val_accs, label='Validation Accuracy', color='red')
    ax2.set_title('Training and Validation Accuracy')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Accuracy (%)')
    ax2.legend()
    ax2.grid(True)    
    
    plt.tight_layout()
    plt.show() 

In [10]:
def run_training(): 

    train_dataset = RadioML18Dataset(mode = 'train')
    valid_dataset = RadioML18Dataset(mode = 'valid')
    test_dataset = RadioML18Dataset(mode = 'test')

    if len(train_dataset) == 0:
        print("Traingin is empty ! ")
        return None 
    
    train_loader = DataLoader(
        train_dataset, 
        batch_size = BATCH_SIZE, 
        shuffle=True, 
        num_workers=NUM_WORKERS, 
        pin_memory=True, 
        drop_last = True 
    )

    valid_loader = DataLoader(
        valid_dataset, 
        batch_size = BATCH_SIZE, 
        shuffle=False, 
        num_workers=NUM_WORKERS, 
        pin_memory=True, 
        #drop_last = True 
    )

    test_loader = DataLoader(
        test_dataset, 
        batch_size = BATCH_SIZE, 
        shuffle=False, 
        num_workers=NUM_WORKERS, 
        pin_memory=True, 
        #drop_last = True 
    )

    print(f"Train batches : {len(train_loader)} \n Valid batches : {len(valid_loader)} \n Test batches : {len(test_loader)}")

    print("Setting up Training")

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)
    scaler = GradScaler() 

    # Training history
    train_losses = []
    train_accuracies = []
    val_losses = []
    val_accuracies = []
    
    best_val_acc = 0.0   

    print(f"✅ Training setup complete")
    print(f"  Optimizer: Adam (lr={LEARNING_RATE})")
    print(f"  Loss function: CrossEntropyLoss")
    print(f"  Scheduler: StepLR (step=10, gamma=0.5)")
    
    # Step 4: Training loop
    print(f"\n🏋️ Step 4: Training for {NUM_EPOCHS} epochs...")
    print("=" * 50)
    
    start_time = time.time()
    
    for epoch in range(NUM_EPOCHS):
        print(f"\nEpoch {epoch+1}/{NUM_EPOCHS}")
        print("-" * 30)
        
        # Train
        train_loss, train_acc = train_one_epoch(
            model, train_loader, criterion, optimizer, scaler, device
        )
        
        # Validate
        val_loss, val_acc = validate_one_epoch(
            model, valid_loader, criterion, device
        )
        
        # Update scheduler
        scheduler.step()
        
        # Save history
        train_losses.append(train_loss)
        train_accuracies.append(train_acc)
        val_losses.append(val_loss)
        val_accuracies.append(val_acc)
        
        # Print results
        print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
        print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")
        
        # Save best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'best_val_acc': best_val_acc,
            }, 'best_model_simple.pth')
            print(f"💾 New best model saved! (Val Acc: {val_acc:.2f}%)")
        
        # Memory cleanup
        torch.cuda.empty_cache()
        gc.collect()
    
    training_time = time.time() - start_time
    print(f"\n⏱️ Training completed in {training_time/60:.2f} minutes")
    
    # Step 5: Test the best model
    print("\n🧪 Step 5: Testing best model...")
    
    # Load best model
    checkpoint = torch.load('best_model_simple.pth', map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    
    # Test
    test_loss, test_acc = validate_one_epoch(model, test_loader, criterion, device)
    
    print(f"🎯 Final Results:")
    print(f"  Best Validation Accuracy: {best_val_acc:.2f}%")
    print(f"  Test Accuracy: {test_acc:.2f}%")
    print(f"  Test Loss: {test_loss:.4f}")
    
    # Step 6: Plot results
    print("\n📈 Step 6: Plotting results...")
    plot_training_results(train_losses, val_losses, train_accuracies, val_accuracies)
    
    results = {
        'train_losses': train_losses,
        'val_losses': val_losses,
        'train_accuracies': train_accuracies,
        'val_accuracies': val_accuracies,
        'best_val_acc': best_val_acc,
        'test_acc': test_acc,
        'training_time': training_time
    }
    
    print("🎉 Training pipeline completed successfully!")
    return results

# Run the training (uncomment to start)
print("🔥 Ready to start training!")
print("💡 Run: results = run_training()")
print("   or just execute this cell to start training immediately")   

🔥 Ready to start training!
💡 Run: results = run_training()
   or just execute this cell to start training immediately


In [11]:
def test_model_performance():
    """
    Detailed testing and visualization of model performance
    """
    print("🧪 Detailed Model Testing")
    print("=" * 40)
    
    # Load test dataset
    test_dataset = RadioML18Dataset(mode='test')
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
    # Load best model
    try:
        checkpoint = torch.load('best_model_simple.pth', map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        print("✅ Best model loaded")
    except:
        print("⚠️ No saved model found, using current model")
    
    model.eval()
    
    # Collect predictions
    all_predictions = []
    all_targets = []
    all_snrs = []
    
    print("🔍 Collecting predictions...")
    
    with torch.no_grad():
        for data, target, snr in tqdm(test_loader, desc="Testing"):
            data = data.to(device)
            output = model(data)
            
            predictions = torch.argmax(output, dim=1).cpu().numpy()
            targets = target.numpy()
            snrs = snr.numpy()
            
            all_predictions.extend(predictions)
            all_targets.extend(targets)
            all_snrs.extend(snrs)
    
    all_predictions = np.array(all_predictions)
    all_targets = np.array(all_targets)
    all_snrs = np.array(all_snrs)
    
    # Calculate overall accuracy
    overall_accuracy = np.mean(all_predictions == all_targets) * 100
    print(f"🎯 Overall Test Accuracy: {overall_accuracy:.2f}%")
    
    # Accuracy per SNR
    unique_snrs = np.unique(all_snrs)
    snr_accuracies = []
    
    print(f"\n📊 Accuracy per SNR:")
    for snr in sorted(unique_snrs):
        mask = all_snrs == snr
        if np.sum(mask) > 0:
            acc = np.mean(all_predictions[mask] == all_targets[mask]) * 100
            snr_accuracies.append(acc)
            print(f"  SNR {snr:3.0f} dB: {acc:5.2f}%")
        else:
            snr_accuracies.append(0)
    
    # Plot accuracy vs SNR
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 2, 1)
    plt.plot(sorted(unique_snrs), snr_accuracies, 'bo-', linewidth=2, markersize=6)
    plt.xlabel('SNR (dB)')
    plt.ylabel('Accuracy (%)')
    plt.title('Classification Accuracy vs SNR')
    plt.grid(True, alpha=0.3)
    plt.ylim(0, 100)
    
    # Confusion matrix
    from sklearn.metrics import confusion_matrix
    import seaborn as sns
    
    cm = confusion_matrix(all_targets, all_predictions)
    
    plt.subplot(1, 2, 2)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=TARGET_MODULATIONS,
                yticklabels=TARGET_MODULATIONS)
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix')
    
    plt.tight_layout()
    plt.show()
    
    # Classification report
    print(f"\n📋 Classification Report:")
    from sklearn.metrics import classification_report
    report = classification_report(all_targets, all_predictions, 
                                   target_names=TARGET_MODULATIONS)
    print(report)
    
    return {
        'overall_accuracy': overall_accuracy,
        'snr_accuracies': dict(zip(sorted(unique_snrs), snr_accuracies)),
        'confusion_matrix': cm,
        'predictions': all_predictions,
        'targets': all_targets,
        'snrs': all_snrs
    }

def quick_test():
    """
    Quick test to verify everything is working
    """
    print("⚡ Quick Test")
    print("=" * 20)
    
    # Create a small test dataset
    test_dataset = RadioML18Dataset(mode='test')
    
    if len(test_dataset) == 0:
        print("❌ No test data available")
        return
    
    # Test a few samples
    print(f"📊 Test dataset: {len(test_dataset):,} samples")
    
    # Get a sample
    sample_x, sample_y, sample_z = test_dataset[0]
    print(f"Sample shape: {sample_x.shape}")
    print(f"Sample label: {sample_y} ({TARGET_MODULATIONS[sample_y]})")
    print(f"Sample SNR: {sample_z} dB")
    
    # Test model prediction
    model.eval()
    with torch.no_grad():
        sample_x = sample_x.unsqueeze(0).to(device)  # Add batch dimension
        output = model(sample_x)
        prediction = torch.argmax(output, dim=1).item()
        confidence = torch.softmax(output, dim=1).max().item()
    
    print(f"Model prediction: {prediction} ({TARGET_MODULATIONS[prediction]})")
    print(f"Confidence: {confidence:.3f}")
    print(f"Correct: {'✅' if prediction == sample_y else '❌'}")
    
    print("✅ Quick test completed!")

In [12]:
quick_test()

⚡ Quick Test
loading test dataset .. 
Target modulation : ['OOK', '4ASK', '8ASK', 'BPSK', 'QPSK', '8PSK', '16PSK', '32PSK']
File loaded Succesfully
Total dataset : 2,555,904 samples and for SNR Range -20 - 30 dB 
found len 8 valid modulations for test split
No data found for test split !
No data found for test split! 
❌ No test data available


In [13]:
result = run_training()

loading train dataset .. 
Target modulation : ['OOK', '4ASK', '8ASK', 'BPSK', 'QPSK', '8PSK', '16PSK', '32PSK']
File loaded Succesfully
Total dataset : 2,555,904 samples and for SNR Range -20 - 30 dB 
found len 8 valid modulations for train split


KeyboardInterrupt: 